In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Reactive
using Interact
using SplineUtils
using NearestNeighbors


import PyPlot

# Types

In [ ]:
type HRHC <: DriverModel{NextState, IntegratedContinuous}
    action_context::IntegratedContinuous
    v_cmds # possible velocity commands
    δ_cmds # possible δ_cmds
    ΔXYθ # state changes associated with cmd = (v_command, δ_command)
    legal_ΔXYθ # reachable from current v, δ
    legal_v # reachable from current v, δ
    legal_δ # reachable from current v, δ
    v_idx_low::Int # index lowest reachable v_command
    v_idx_high::Int # index highest reachable v_command
    δ_idx_low::Int # index lowest reachable δ_command
    δ_idx_high::Int # index highest reachable δ_command
    successor_states # reachable next_states
    
    #car parameters with bicycle geometry model
    car_length::Float64 # wheel base
    car_width::Float64
    car_ID::Int
    
    # current v, current δ
    v::Float64
    δ::Float64
    curve_ind::Int
    Δs::Float64
    
    # planning horizon
    h::Int
    Δt::Float64
    
    # logit level
    k::Int
    
    # reachable range of V and δ within a single time step
    ΔV₊::Float64
    ΔV₋::Float64
    Δδ::Float64
    
    V_MIN::Float64
    V_MAX::Float64
    V_STEPS::Int
    δ_MAX::Float64
    δ_MIN::Float64
    δ_STEPS::Int

    # maximum deviation from center of track (if |t| > T_MAX, car is out of bounds)
    T_MAX::Float64
    
    # Action = Next State
    action::NextState
    
    function HRHC(car_ID::Int,roadway,context;
        car_length::Float64=4.8,
        car_width::Float64=2.5,
        v::Float64=0.0,
        δ::Float64=0.0,
        h::Int=10,
        Δt::Float64=1.0/24,
        ΔV₊::Float64=1.55,
        ΔV₋::Float64=3.05,
        Δδ::Float64=Float64(π)/12,
        V_MIN::Float64=0.0,
        V_MAX::Float64=100.0,
        V_STEPS::Int=101,
        δ_MAX::Float64=Float64(π)/8,
        δ_MIN::Float64=-Float64(π)/8,
        δ_STEPS::Int=16,
        k::Int=1
        )
        
        hrhc = new()
        
        hrhc.V_MIN=V_MIN
        hrhc.V_MAX=V_MAX
        hrhc.V_STEPS=V_STEPS
        hrhc.δ_MAX=δ_MAX
        hrhc.δ_MIN=δ_MIN
        hrhc.δ_STEPS=δ_STEPS
        hrhc.T_MAX=(roadway.segments[1].lanes[1].width - car_width)/2.0
        
        hrhc.car_ID = car_ID
        hrhc.car_length=car_length
        hrhc.car_width=car_width
        hrhc.h=h
        hrhc.Δt=Δt
        hrhc.ΔV₊=ΔV₊
        hrhc.ΔV₋=ΔV₋
        hrhc.Δδ=Δδ
        
        hrhc.k=k
        
        hrhc.v_cmds, hrhc.δ_cmds, hrhc.ΔXYθ = MotionPrimitives(car_length,car_width,h,Δt,V_MIN,V_MAX,V_STEPS,δ_MAX,δ_MIN,δ_STEPS)
        QuadraticMask!(hrhc.ΔXYθ)
        
        hrhc.v=v
        hrhc.δ=δ
        hrhc.curve_ind=1
        hrhc.Δs=roadway.segments[1].lanes[1].curve[2].s-roadway.segments[1].lanes[1].curve[1].s
        hrhc.action_context=context    
        hrhc.action = NextState(VehicleState(VecSE2(0,0,0),0.0))
        
        hrhc
    end
end

get_name(::HRHC) = "HRHC"
action_context(driver::HRHC) = driver.action_context # AutomotiveDrivingModels.action_context
Base.rand(hrhc::HRHC) = hrhc.action


# utility functions

In [ ]:
function GenSplineRoadway(x,y,θ,s,k,lane_width)
    nlanes = 1
    seg1 = RoadSegment(1, Array(Lane, nlanes))
    tag1=LaneTag(1,1)
    boundary_left = LaneBoundary(:solid, :white)
    boundary_right = LaneBoundary(:solid, :white)

    curvepts = Array(CurvePt, length(x))
    for i in 1:length(x)
        curvepts[i] = CurvePt(VecSE2(x[i],y[i],θ[i]), s[i], k[i], NaN)
    end

    curveind_lo = CurveIndex(1,0.0)
    curveind_hi = CurveIndex(length(curvepts)-1,1.0)
#     curveind_hi = CurveIndex(length(curvepts),1.0)

    seg1.lanes[1] = Lane(tag1, curvepts, width=lane_width,
                                  boundary_left=boundary_left, boundary_right=boundary_right,
                                  next = RoadIndex(curveind_lo, tag1),
                                  prev = RoadIndex(curveind_hi, tag1),
                                 )
    
    roadway = Roadway()
    push!(roadway.segments, seg1)
    
    return roadway
end

In [ ]:
# Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
function MotionPrimitives(car_length,car_width,h,Δt,v_min,v_max,v_steps,δ_max,δ_min,δ_steps)
    # 3D array to store motion_primitives
    motion_primitives = zeros(v_steps,2*δ_steps+1,2) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,v_steps)*ones(1,2*δ_steps+1)
    δ = (linspace(-δ_max,δ_max,δ_steps*2+1)*ones(1,v_steps))' # steering angle

    motion_primitives[:,:,1] = v*Δt*h # arc length = velocity * time
    motion_primitives[:,:,2] = car_length./sin(δ) # radius of curvature (+ or -)
    motion_primitives[:,1+δ_steps,2] = Inf; # radius of curvature is infinite if steering angle = 0
    
    destination_primitives = zeros(v_steps,2*δ_steps+1,h,3) # lookup table defining car's location at each of next h time steps

    for i = 1:h
        # angle = 2π * arc_length / r
        dθ = v*Δt*i ./ motion_primitives[:,:,2]
        
        # dX = abs(radius) * sin(angle)
        destination_primitives[:,:,i,1] = abs(motion_primitives[:,:,2]) .* sin(abs(dθ))
        destination_primitives[:,1+δ_steps,i,1] = v[:,1+δ_steps]*Δt*i # centerline
        
        # dY = radius * (1 - cos(angle))
        destination_primitives[:,:,i,2] = motion_primitives[:,:,2].*(1 - cos(dθ))
        destination_primitives[:,1+δ_steps,i,2] = 0 # centerline
        
        destination_primitives[:,:,i,3] = dθ
    end
    
    # motion_primitives[v, δ, i,j] = s (arc_length), r (radius of curvature)
    # destination_primitives[v, δ, h, 1=Δx,2=Δy,3=Δθ]= changes in x, y and θ after h time steps
    return v, δ, destination_primitives

end

In [ ]:
# quadratic logical mask
function QuadraticMask!(library)
    A = size(library)[1]
    B = size(library)[2]
    X = linspace(0,100,A)*ones(1,B)
    Y = ones(A,1)*linspace(-20,20,B)'
    f = X.^2 + 24*(Y.^2 - 10^2)
    mask = 1.0*(f.<10000)
    

#     p = PyPlot.scatter(X,Y.*mask)
#     PyPlot.xlabel("velocity")
#     PyPlot.ylabel("steering angle")
#     PyPlot.show()

    if length(size(library)) == 3
        for i in 1 : size(library)[3]
            library[:,:,i] = library[:,:,i] .* mask
        end
    end
    if length(size(library)) == 4
        for i in 1 : size(library)[3]
            for j in 1 : size(library)[4]
                library[:,:,i,j] = library[:,:,i,j] .* mask
            end
        end
    end
end

In [ ]:
function CurveDist(pt1::CurvePt, pt2::CurvePt)
#     Δx = pt1.pos.x - pt2.pos.x
#     Δy = pt1.pos.y - pt2.pos.y
#     d = sqrt(Δx^2 + Δy^2)
    d = sqrt((pt1.pos.x - pt2.pos.x)^2 + (pt1.pos.y - pt2.pos.y)^2)
end

In [ ]:
function wrap_to_π(θ)
#     θ = θ - div(θ,2*Float64(π))*(2*Float64(π))
    θ = θ - div(θ,2*Float64(π))*(2*Float64(π)) 
    θ = θ + (θ .< -π).*(2*Float64(π)) - (θ .> π).*(2*Float64(π))
end

# Plotting Functions

# HRHC Algorithm

### 0. Initialize motion primitives

### 1. Identify eligible (reachable) motion primitives based on current state

In [ ]:
function getLegalMoves!(hrhc::HRHC, scene; h=hrhc.h)
    v_norm = scene.vehicles[hrhc.car_ID].state.v / hrhc.V_MAX

    # Restrict search space to reachable states
    hrhc.v_idx_low = max(1, round(Int,(v_norm - hrhc.ΔV₋/hrhc.V_MAX)*hrhc.V_STEPS)) # index of lowest reachable v in the next time step
    hrhc.v_idx_high = min(hrhc.V_STEPS, 1+round(Int, (v_norm + hrhc.ΔV₊/hrhc.V_MAX)*hrhc.V_STEPS)) # highest reachable v in the next time step

    # Restrict search space to reachable states
    hrhc.δ_idx_low = max(1, (hrhc.δ_STEPS+1) + round(Int,((hrhc.δ - hrhc.Δδ)/(hrhc.δ_MAX - hrhc.δ_MIN))*(2*hrhc.δ_STEPS+1)))
    hrhc.δ_idx_high = min((2*hrhc.δ_STEPS+1), (hrhc.δ_STEPS+1) + round(Int,((hrhc.δ + hrhc.Δδ)/(hrhc.δ_MAX - hrhc.δ_MIN))*(2*hrhc.δ_STEPS+1)))

    # legal_moves = motion_primitives[v_idx_low:v_idx_high,δ_idx_low:δ_idx_high,:]
    hrhc.legal_ΔXYθ = hrhc.ΔXYθ[hrhc.v_idx_low:hrhc.v_idx_high,hrhc.δ_idx_low:hrhc.δ_idx_high,h,:] # ΔX, ΔY, Δθ
    hrhc.legal_v = hrhc.v_cmds[hrhc.v_idx_low:hrhc.v_idx_high,hrhc.δ_idx_low:hrhc.δ_idx_high]
    hrhc.legal_δ = hrhc.δ_cmds[hrhc.v_idx_low:hrhc.v_idx_high,hrhc.δ_idx_low:hrhc.δ_idx_high]
    
    return
end

### 2. Calculate resulting final state (posG) from applying each given motion primitive

In [ ]:
#function getSuccessorStates(vehicle, hrhc)
function getSuccessorStates!(hrhc::HRHC, scene::Scene)
    pos = scene.vehicles[hrhc.car_ID].state.posG # global x,y,z of car

    ΔX = hrhc.legal_ΔXYθ[:,:,1] * cos(pos.θ) + hrhc.legal_ΔXYθ[:,:,2] * -sin(pos.θ)
    ΔY = hrhc.legal_ΔXYθ[:,:,1] * sin(pos.θ) + hrhc.legal_ΔXYθ[:,:,2] * cos(pos.θ)
    Δθ = hrhc.legal_ΔXYθ[:,:,3]

    hrhc.successor_states = zeros(size(hrhc.legal_ΔXYθ))
    hrhc.successor_states[:,:,1] = ΔX + pos.x
    hrhc.successor_states[:,:,2] = ΔY + pos.y
    hrhc.successor_states[:,:,3] = Δθ + pos.θ
    
    return
end

### 3. Project posG onto track to determine if a given motion is legal (in bounds)

In [ ]:
function loopProjectionKD(hrhc::HRHC,scene,roadway,tree)
    curve = roadway.segments[1].lanes[1].curve

    s_grid = zeros(size(hrhc.successor_states,1),size(hrhc.successor_states,2))
    t_grid = zeros(size(s_grid))
    ϕ_grid = zeros(size(s_grid))
    idx_grid = zeros(Int,size(s_grid))
    
    pts = [reshape(hrhc.successor_states[:,:,1],size(hrhc.successor_states[:,:,1],1)*size(hrhc.successor_states[:,:,1],2),1)';
    reshape(hrhc.successor_states[:,:,2],size(hrhc.successor_states[:,:,2],1)*size(hrhc.successor_states[:,:,2],2),1)']
    idxs_list, _ = knn(tree,pts,1)
    idxs=reshape(idxs_list,size(hrhc.successor_states[:,:,2],1),size(hrhc.successor_states[:,:,2],2))
    
    
    for i in 1:size(s_grid,1)
        for j in 1:size(s_grid,2)
            idxA = idxs[i,j][1]-1
            idxB = idxs[i,j][1]+1
            if idxs[i,j][1] == length(curve)
                idxB = 1 # wrap to the beginning of the curve
            end
            if idxs[i,j][1] == 1
                idxA = length(curve) # wrap to the end of the curve
            end
            x = hrhc.successor_states[i,j,1]
            y = hrhc.successor_states[i,j,2]
            dA = sqrt(sum(([curve[idxA].pos.x, curve[idxA].pos.y]-[x,y]).^2))
            dB = sqrt(sum(([curve[idxB].pos.x, curve[idxB].pos.y]-[x,y]).^2))
            if dA < dB
                idxB = idxs[i,j][1]
            else
                idxA = idxs[i,j][1]
            end
            
            # project
            vec1 = [curve[idxB].pos.x - curve[idxA].pos.x, curve[idxB].pos.y - curve[idxA].pos.y, 0]
            vec2 = [x - curve[idxA].pos.x, y - curve[idxA].pos.y, 0]
            idx_t = dot(vec2, vec1)/norm(vec1)^2
            
            s_θ = curve[idxA].pos.θ + idx_t*(curve[idxB].pos.θ - curve[idxA].pos.θ)
    
            s_grid[i,j] = curve[idxA].s + idx_t*hrhc.Δs
            t_grid[i,j] = norm(vec2 - idx_t*vec1)*sign(sum(cross(vec1, vec2)))
            ϕ_grid[i,j] = wrap_to_π(hrhc.successor_states[i,j,3] - s_θ)
            idx_grid[i,j] = idxA
        end
    end
    # account for wrap-around
    s_grid[s_grid .< scene.vehicles[hrhc.car_ID].state.posF.s] += curve[end].s + hrhc.Δs
    
    return s_grid, t_grid, ϕ_grid
end

In [ ]:
function kdProject(x,y,θ,tree,roadway,hrhc)
    curve = roadway.segments[1].lanes[1].curve

    idx_list,dist = knn(tree,[x;y],1)
    idx = idx_list[1]
    idxA = idx-1
    idxB = idx+1

    if idx == length(curve)
        idxB = 1 # back to the beginning of the curve
    end
    if idx == 1
        idxA = length(curve)
    end
    dA = sqrt(sum(([curve[idxA].pos.x, curve[idxA].pos.y]-[x,y]).^2))
    dB = sqrt(sum(([curve[idxB].pos.x, curve[idxB].pos.y]-[x,y]).^2))
    if dA < dB
        idxB = idx
    else
        idxA = idx
    end

    # project
    vec1 = [curve[idxB].pos.x - curve[idxA].pos.x, curve[idxB].pos.y - curve[idxA].pos.y, 0]
    vec2 = [x - curve[idxA].pos.x, y - curve[idxA].pos.y, 0]
    idx_t = dot(vec2, vec1)/norm(vec1)^2

    pθ = curve[idxA].pos.θ + idx_t*(curve[idxB].pos.θ - curve[idxA].pos.θ)

    s = curve[idxA].s + idx_t*hrhc.Δs
    t = -norm(vec2 - idx_t*vec1)
    ϕ = wrap_to_π(θ - pθ)

    s,t,ϕ,idxA
end

### 4. Select legal motion primitive that yields highest progress

### 5. Check each time step up to horizon to make sure the entire trajectory is legal

In [ ]:
function applyObstacleKernel!(arr, obstacle_ID, hrhc, scene, roadway)
    """
    computes a penalty function over hrhc.successor_states to penalize which
    successor states will result in a collision with the obstacle
    """
    
    pos = scene.vehicles[obstacle_ID].state.posG # position of obstacle
    x = pos.x
    y = pos.y
    
    r = sqrt((hrhc.successor_states[:,:,1] - x).^2 + (hrhc.successor_states[:,:,2] - y).^2)
    
    kernel = 1.0*(r .> hrhc.car_length)
    arr = arr .* kernel
end

In [ ]:
function computeTrajectory(hrhc::HRHC, scene, cmd_index; h=hrhc.h)
    pos = scene.vehicles[hrhc.car_ID].state.posG
    
    traj_ΔXYθ = hrhc.ΔXYθ[cmd_index[1],cmd_index[2],1:h,:]
    
    ΔX = traj_ΔXYθ[:,1] * cos(pos.θ) + traj_ΔXYθ[:,2] * -sin(pos.θ)
    ΔY = traj_ΔXYθ[:,1] * sin(pos.θ) + traj_ΔXYθ[:,2] * cos(pos.θ)
    Δθ = traj_ΔXYθ[:,3]

    trajectory = zeros(size(traj_ΔXYθ,1),size(traj_ΔXYθ,2)+2)
    trajectory[:,1] = ΔX + pos.x
    trajectory[:,2] = ΔY + pos.y
    trajectory[:,3] = Δθ + pos.θ
    trajectory[:,4] = hrhc.v_cmds[cmd_index[1],cmd_index[2]]
    trajectory[:,5] = hrhc.δ_cmds[cmd_index[1],cmd_index[2]]
    
    return trajectory
end

In [ ]:
function screenTrajectory(trajectory, obstacleMap, scene, roadway, hrhc, tree, k_level)
    out_of_bounds = false
    collision_flag = false
    threshold_dist = 4.0*hrhc.car_length
    for i in 1 : size(trajectory,1)
        x = trajectory[i,1]
        y = trajectory[i,2]
        θ = trajectory[i,3]
        s,t,ϕ = kdProject(x,y,θ,tree,roadway,hrhc)

    #     if k_level >= 1
    #         for (id,car) in obstacleMap[k_level - 1]
    #             if id != hrhc.car_ID
    #                 state = scene.vehicles[hrhc.car_ID].state
    #                 state2 = scene.vehicles[id].state
    #                 diff = state.posG - state2.posG
    #                 if norm([diff.x, diff.y]) < threshold_dist
    #                     egoCar = VecSE2(x,y,θ)
    #                     car2 = VecSE2(car[i,1],car[i,2],car[i,3])
    #                     collision_flag = checkCollision(scene,egoCar,car2,hrhc.car_ID,id)
    #                     if collision_flag
    #                         return collision_flag
    #                     end
    #                 end
    #             end
    #         end
    #     end
        
        if abs(t) > hrhc.T_MAX
            out_of_bounds=true
            return out_of_bounds
        end
    end
    return out_of_bounds
end

In [ ]:
obstacleMap[1][1]

In [ ]:
function checkCollision(scene,car1::VecSE2,car2::VecSE2,id1,id2)
    L1 = scene.vehicles[id1].def.length
    w1 = scene.vehicles[id1].def.width
    L2 = scene.vehicles[id2].def.length
    w2 = scene.vehicles[id2].def.width
    diff = car1 - car2
    R = [cos(diff.θ) -sin(diff.θ); sin(diff.θ) cos(diff.θ)]
    car2_pts = R*[L2 L2 -L2 -L2; -w2 w2 w2 -w2]/2.0 + [diff.x 0;0 diff.y]*ones(2,4)
    collision_flag = (sum((abs(car2_pts[1,:]) .>= L1/2.0).*(abs(car2_pts[2,:]) .>= w1/2.0)) > 0)
    return collision_flag
end

### 6. Apply the motion primitive for a single time step

### 7. Go back to 1

## Obstacle Map

In [ ]:
function generateObstacleMap(scene, models)
    k = maximum([driver.k for (id, driver) in models])
    n = length(scene)
    h = maximum([driver.h for (id, driver) in models]) # h should be the same for all vehicles on the track
    obstacleDict = Dict()
    for level in 0:k
        obstacleDict[level] = Dict()
        for (id, driver) in models
            obstacleDict[level][id] = zeros(h,5) # x,y,θ,v,δ
        end
    end
    
    return obstacleDict
end

In [ ]:
function updateObstacleMap!(obstacleMap, level, car_ID, trajectory)
    obstacleMap[level][car_ID][1:size(trajectory,1),1:size(trajectory,2)] = trajectory
end

In [ ]:
function getObstacleCoords(obstacleMap, level, car_ID, h)
    return obstacleMap[level][car_ID][h,:]
end

# get HRHC action function

In [ ]:
function calculateObjective(hrhc, scene, roadway, tree, s, t, ϕ, obstacleMap, k_level, h; f_ϕ=0.0, f_t=0.1, f_tϕ=3.0)
    # optimization objective
    state = scene.vehicles[hrhc.car_ID].state
    dS = s - state.posF.s
    dS = dS / maximum(dS) # normalize
    ϕMAX = Float64(π)/2
    
    # penalize large t (proximity to edge of track)
    cost_t = (exp(((10-h+f_t)*abs(t/hrhc.T_MAX).^2)) - 1)/exp(f_t)
    # penalize large ϕ (steering away from forward direction on the track)
    cost_ϕ = (exp(((10-h+f_ϕ)*abs(ϕ/ϕMAX).^2)) - 1)/exp(f_ϕ)
    # penalize when t and ϕ have the same sign
    A = [1 1; 1 1]
    cost_x = (((ϕ/ϕMAX)*A[1,1] + (t/T_MAX)*A[2,1]).*(ϕ/ϕMAX) + ((ϕ/ϕMAX)*A[1,2] + (t/T_MAX)*A[2,2]).*(t/T_MAX))/2
    cost_tϕ = (exp(f_tϕ*cost_x) - 1)/exp(1)
    eligibility_mask = ((hrhc.successor_states[:,:,1] .== state.posG.x).*(hrhc.successor_states[:,:,2] .== state.posG.y))
    
    # obstacles
    collisionCost = zeros(size(cost_t))
    threshold_dist = hrhc.car_length*4 # must be at least this close before we care to calculate collision cost
    if k_level >= 1
        for (id,car) in obstacleMap[k_level - 1]
            if id != hrhc.car_ID
                state = scene.vehicles[hrhc.car_ID].state
                state2 = scene.vehicles[id].state
                diff = state.posG - state2.posG
                if norm([diff.x, diff.y]) < threshold_dist
                    pos = VecSE2(car[h,1:3]) # x,y,θ of opponent at time step h                              
                    ΔX = hrhc.successor_states[:,:,1] - pos.x # Δx, with opponent at origin
                    ΔY = hrhc.successor_states[:,:,2] - pos.y # Δy with opponent at origin
                    Δθ = hrhc.successor_states[:,:,3] - pos.θ # Δθ with opponent at origin
                    pts = [hrhc.car_length hrhc.car_length -hrhc.car_length -hrhc.car_length;
                        -hrhc.car_width hrhc.car_width hrhc.car_width -hrhc.car_width]/2.0
                    pX = zeros(4,size(hrhc.successor_states,1),size(hrhc.successor_states,2))
                    pY = zeros(size(pX))
                    for i in 1:4
                        pX[i,:,:] = pts[1,i]*cos(Δθ) - pts[2,i]*sin(Δθ) + ΔX
                        pY[i,:,:] = pts[1,i]*sin(Δθ) + pts[2,i].*cos(Δθ) + ΔY
                    end

                    collisionFlag = (sum((abs(pX) .< hrhc.car_length/2.0),1)[1,:,:]).*(sum((abs(pY) .< hrhc.car_width/2.0),1)[1,:,:])
                    collisionCost = .001+(collisionFlag .>= 1)./(minimum(abs(pX),1)[1,:,:].*minimum(abs(pY),1)[1,:,:])
                end
            end
        end
    end
    
    objective = cost_t + cost_ϕ + cost_tϕ + 1 - dS + collisionCost + Inf * eligibility_mask
    return objective
end

In [ ]:
function observe!(hrhc::HRHC, scene::Scene, roadway::Roadway, egoid::Int, tree::KDTree, obstacleMap, k_level)
    if k_level > hrhc.k
        return
    end
    state = scene.vehicles[hrhc.car_ID].state
    hrhc.curve_ind = state.posF.roadind.ind.i
    v = state.v # current v
    hrhc.v = v
    
    trajectory = zeros(hrhc.h,5)
    action_selected = false
    abs_cmd = (1,1)
    
    i = 0
    for i in 0:(hrhc.h-1)
        if action_selected
            break # out of for loop
        end
        
        # get legal (reachable from current v, δ) actions
        getLegalMoves!(hrhc, scene, h=hrhc.h-i)
        
        # calculate successor states
        getSuccessorStates!(hrhc, scene)
        
        # project successor states onto track
        s,t,ϕ = loopProjectionKD(hrhc, scene, roadway, tree)
        
        # optimization objective        
        objective = calculateObjective(hrhc, scene, roadway, tree, s, t, ϕ, obstacleMap, k_level, hrhc.h-i,f_t=0.0)

        while (action_selected==false) && (minimum(objective) != Inf)            
            index = indmin(objective) # find get a better method of optimizing this
            cmd = ind2sub(s, index)
            abs_cmd = (cmd[1]+hrhc.v_idx_low-1, cmd[2]+hrhc.δ_idx_low-1)

            # compute full trajectory up to horizon
            trajectory = computeTrajectory(hrhc, scene, abs_cmd, h=hrhc.h-i)
            
            # screen trajectory for collisions / validity
            out_of_bounds = screenTrajectory(trajectory, obstacleMap, scene, roadway, hrhc, tree, k_level)
            
            if out_of_bounds
                objective[index] = Inf
            else
                action_selected=true
                updateObstacleMap!(obstacleMap, k_level, hrhc.car_ID, trajectory)
            end
        end 
    end
    
    hrhc.δ = hrhc.δ_cmds[abs_cmd[1], abs_cmd[2]]
    hrhc.v = hrhc.v_cmds[abs_cmd[1], abs_cmd[2]]
    
    next_state = VehicleState(VecSE2(trajectory[1,1:3]),roadway,hrhc.v)   
    hrhc.action = NextState(next_state) # action
end

In [ ]:
s,t,ϕ = loopProjectionKD(models[2], scene, roadway, tree)
# objective, collisionCost = calculateObjective(models[2], scene, roadway, tree, s, t, ϕ, obstacleMap, 0, models[2].h, f_t=0.0)
# PyPlot.pcolor(collisionCost)


In [ ]:
function plotSplineRoadway(x,y,θ,lane_width)
    perp_lines1 = zeros(2,length(x))
    perp_lines2 = zeros(2,length(x))
    
    perp_lines1[1,:] = x + (lane_width/2.0)*sin(θ)
    perp_lines1[2,:] = y - (lane_width/2.0)*cos(θ)
    perp_lines2[1,:] = x - (lane_width/2.0)*sin(θ)
    perp_lines2[2,:] = y + (lane_width/2.0)*cos(θ)
    
#     PyPlot.figure()
#     PyPlot.scatter(x,y)
    PyPlot.plot(x,y)
    PyPlot.plot(perp_lines1[1,:],perp_lines1[2,:],color="green")
    PyPlot.plot(perp_lines2[1,:],perp_lines2[2,:],color="green")
    PyPlot.axis("equal")
#     PyPlot.show()
end

## Initialize Parameters

In [ ]:
# scene parameters
scene = Scene()
framerate = 24
Δt = 1.0/framerate # size of rendering timesteps
n_integration_sub_steps = 3 # sub steps for smoother integration
context = IntegratedContinuous(Δt, n_integration_sub_steps) # integrated continuous context


#car parameters
car_length = 4.8 # front wheel to back wheel
car_width = 2.5
v⁰  = 0.0 # initial velocity
δ⁰ = 0.0 # initical steering angle

############### INITIALIZE TRACK ################

lane_width = 8.0*DEFAULT_LANE_WIDTH
radius = 45.0
edge_buffer = 0.25
T_MAX = lane_width/2.0 - car_width/2.0 - edge_buffer # max allowable projection distance from center of lane
base_speed= 0.0

# spline control points
# Pts = 40*[0 -1 -2 -3 -3 -3 -2 -1 -1 -1 -2 -3 -4 -5 -5 -5 -5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
#          0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]
Pts = 40*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -5.5 -5 -4.5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]


degree = 3 # degree of spline
num_points = 10001

# compute B spline
T, tt, rx, ry = ClosedB_Spline(Pts, degree, num_points)
# compute B spline derivative
ṙx, ṙy = B_SplineDerivative(T,tt,Pts,degree)
# compute theta 
θ = atan2(ṙy,ṙx) # unit tangent vector
# compute arc length s
s = zeros(size(rx))
s[2:end] = cumsum(sqrt(diff(rx).^2 + diff(ry).^2))
# compute curvature
k = diff(θ)./diff(s) # curvature
# resample evenly along spline
num_samples = 420
xP, yP, θP, sP, kP = ResampleSplineEven(rx,ry,θ,s,k,num_samples)

# KDtree for projection onto roadway
tree = KDTree([xP';yP'])

lane_width = 20.0
roadway = GenSplineRoadway(xP,yP,θP,sP,kP,lane_width)

roadind1 = RoadIndex(CurveIndex(8,0),LaneTag(1,1))
roadind2 = RoadIndex(CurveIndex(5,0),LaneTag(1,1))
roadind3 = RoadIndex(CurveIndex(1,0),LaneTag(1,1))

vehstate1 = VehicleState(Frenet(roadind1, roadway), roadway, 0.0)
vehstate2 = VehicleState(Frenet(roadind2, roadway), roadway, 0.0)
vehstate3 = VehicleState(Frenet(roadind3, roadway), roadway, 0.0)

vehdef1 = VehicleDef(1,AgentClass.CAR, car_length, car_width)
vehdef2 = VehicleDef(2,AgentClass.CAR, car_length, car_width)
vehdef3 = VehicleDef(3,AgentClass.CAR, car_length, car_width)

push!(scene,Vehicle(vehstate1, vehdef1))
push!(scene,Vehicle(vehstate2, vehdef2))
push!(scene,Vehicle(vehstate3, vehdef3))

models = Dict{Int, DriverModel}()
carcolors = Dict{Int, Colorant}()
carcolors[1] = colorant"red"
carcolors[2] = colorant"blue"
carcolors[2] = colorant"yellow"


# hrhc = HRHC(1,roadway,context)
models[1] = HRHC(1,roadway,context,ΔV₊=2.05,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=115.0,V_STEPS=230,k=1)
models[2] = HRHC(2,roadway,context,ΔV₊=1.55,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=125.0,V_STEPS=250,k=1)
models[3] = HRHC(3,roadway,context,ΔV₊=2.55,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=130.0,V_STEPS=250,k=2)
actions = Array(DriveAction, length(scene))

obstacleMap = generateObstacleMap(scene, models)

# Run HRHC:

In [ ]:
function drawsim(t, dt=NaN)
#     get_actions!(actions, scene, roadway, models)
    k_level = 0 # needs to be updated into a loop
    for k_level in 0:maximum([model.k for (id,model) in models])
        for (i, veh) in enumerate(scene)
            model = models[veh.def.id]
            observe!(model, scene, roadway, veh.def.id, tree, obstacleMap, k_level)
            actions[i] = rand(model)
        end
    end
#     tick!(scene, roadway, actions, models)
    for (veh, action) in zip(scene, actions)
        model = models[veh.def.id]
        context = action_context(model)
        veh.state = propagate(veh, action, context, roadway)
    end
    render(scene, roadway, cam=FitToContentCamera(), car_colors=carcolors)
end

In [ ]:
ticks = fps(framerate)
timestamps = map(_ -> time(), ticks)
map(drawsim, timestamps)

In [ ]:
vehstate1 = VehicleState(VecSE2(1,3,Float64(π)/8.0), roadway, 73.0)
vehstate2 = VehicleState(VecSE2(4,0,0), roadway, 80.0)
vehdef1 = VehicleDef(1,AgentClass.CAR, car_length, car_width)
vehdef2 = VehicleDef(2,AgentClass.CAR, car_length, car_width)
veh1 = Vehicle(vehstate1, vehdef1)
veh2 = Vehicle(vehstate2, vehdef2)

# difference between vehicle centerpoints
diff = veh2.state.posG -  veh1.state.posG
R = [cos(diff.θ) -sin(diff.θ);
    sin(diff.θ) cos(diff.θ)]

v1_pts = [car_length/2 car_length/2 -car_length/2 -car_length/2 car_length/2; -car_width/2 car_width/2 car_width/2 -car_width/2 -car_width/2]
v2_pts = R*v1_pts + [diff.x 0;0 diff.y]*ones(2,5)
# collision_flag = 0
collision_flag = (sum((abs(v2_pts[1,:]) .>= car_length/2.0).*(abs(v2_pts[2,:]) .>= car_width/2.0)) > 0)
PyPlot.figure(figsize=[3,2])
for i in 1:4
    PyPlot.plot([v2_pts[1,i],v2_pts[1,i]],[0,v2_pts[2,i]],"green")
end

PyPlot.scatter(v1_pts[1,:],v1_pts[2,:],s=80)
PyPlot.plot(v1_pts[1,:],v1_pts[2,:])
PyPlot.scatter(v2_pts[1,:],v2_pts[2,:],c="red",s=80)
PyPlot.plot(v2_pts[1,:],v2_pts[2,:],c="red")

PyPlot.axis("equal")

# function checkCollision(scene,car1::VecSE2,car2::VecSE2,id1,id2)
#     L1 = scene.vehicles[id1].def.length
#     w1 = scene.vehicles[id1].def.width
#     L2 = scene.vehicles[id2].def.length
#     w2 = scene.vehicles[id2].def.width
#     diff = car1 - car2
#     R = [cos(diff.θ) -sin(diff.θ); sin(diff.θ) cos(diff.θ)]
#     car2_pts = R*[L2 L2 -L2 -L2; -w2 w2 w2 -w2]/2.0 + [diff.x 0;0 diff.y]*ones(2,4)
#     collision_flag = (sum((abs(car2_pts[1,:]) .>= L1/2.0).*(abs(car2_pts[2,:]) .>= w1/2.0)) > 0)
#     return collision_flag
# end

checkCollision(scene,VecSE2(1,3,Float64(π)/8.0), VecSE2(4,0,0),1,2)

In [ ]:
pos = VecSE2(-53.9,-118,.5)   
hrhc = models[1]
ΔX = hrhc.successor_states[:,:,1] - pos.x # Δx, with opponent at origin
ΔY = hrhc.successor_states[:,:,2] - pos.y # Δy with opponent at origin
Δθ = hrhc.successor_states[:,:,3] - pos.θ # Δθ with opponent at origin
pts = [hrhc.car_length hrhc.car_length -hrhc.car_length -hrhc.car_length;
    -hrhc.car_width hrhc.car_width hrhc.car_width -hrhc.car_width]/2.0
pX = zeros(4,size(hrhc.successor_states,1),size(hrhc.successor_states,2))
pY = zeros(size(pX))
for i in 1:4
    pX[i,:,:] = pts[1,i]*cos(Δθ) - pts[2,i]*sin(Δθ) + ΔX
    pY[i,:,:] = pts[1,i]*sin(Δθ) + pts[2,i].*cos(Δθ) + ΔY
end

threshold_dist = hrhc.car_length*4 # must be at least this close before we care to calculate collision cost
collisionFlag = (sum((abs(pX) .< hrhc.car_length/2.0),1)[1,:,:]).*(sum((abs(pY) .< hrhc.car_width/2.0),1)[1,:,:])
collisionCost = .001+(collisionFlag .>= 1)./(minimum(abs(pX),1)[1,:,:].*minimum(abs(pY),1)[1,:,:])
PyPlot.figure(figsize=[3,2])
PyPlot.pcolor(log(collisionCost))
PyPlot.ylabel("v")
PyPlot.xlabel("delta")
collisionFlag

## Test HRHC

In [ ]:
k_level = 1
hrhc = models[1]

state = scene.vehicles[hrhc.car_ID].state
hrhc.curve_ind = state.posF.roadind.ind.i

v = state.v # current v
hrhc.v = v

trajectory = zeros(hrhc.h,5)
out_of_bounds = false
objective = zeros(6,17)
eligibility_mask = zeros(6,17)
s = zeros(6,17)
t = zeros(6,17)
ϕ = zeros(6,17)
action_selected = false
cmd = (1,1)
abs_cmd = (1,1)

i = 0

# get legal (reachable from current v, δ) actions
getLegalMoves!(hrhc, scene, h=hrhc.h-i)

# calculate successor states
getSuccessorStates!(hrhc, scene)

# project successor states onto track
s,t,ϕ = loopProjectionKD(hrhc, scene, roadway, tree)

In [ ]:
f_t = .1
f_ϕ = 0.0
f_tϕ = .3
h  = hrhc.h
# optimization objective
state = scene.vehicles[hrhc.car_ID].state
dS = s - state.posF.s
dS = dS / maximum(dS) # normalize
ϕMAX = Float64(π)/2

# penalize large t (proximity to edge of track)
cost_t = (exp(((10-hrhc.h+f_t)*abs(t/hrhc.T_MAX).^2)) - 1)/exp(f_t)
# penalize large ϕ (steering away from forward direction on the track)
cost_ϕ = (exp(((10-hrhc.h+f_ϕ)*abs(ϕ/ϕMAX).^2)) - 1)/exp(f_ϕ)
# penalize when t and ϕ have the same sign
A = [1 1; 1 1]
cost_x = (((ϕ/ϕMAX)*A[1,1] + (t/T_MAX)*A[2,1]).*(ϕ/ϕMAX) + ((ϕ/ϕMAX)*A[1,2] + (t/T_MAX)*A[2,2]).*(t/T_MAX))/2
cost_tϕ = (exp(f_tϕ*cost_x) - 1)/exp(1)
eligibility_mask = ((hrhc.successor_states[:,:,1] .== state.posG.x).*(hrhc.successor_states[:,:,2] .== state.posG.y))

# obstacles
collisionCost = zeros(size(cost_t))
threshold_dist = hrhc.car_length*4 # must be at least this close before we care to calculate collision cost
if k_level >= 1
    for (id,car) in obstacleMap[k_level - 1]
        if id != hrhc.car_ID
            state = scene.vehicles[hrhc.car_ID].state
            state2 = scene.vehicles[id].state
            diff = state.posG - state2.posG
            if norm([diff.x, diff.y]) < threshold_dist
                pos = VecSE2(car[h,1:3]) # x,y,θ of opponent at time step h                              
                # ΔX = hrhc.successor_states[:,:,1] - pos.x # Δx, with opponent at origin
                # ΔY = hrhc.successor_states[:,:,2] - pos.y # Δy with opponent at origin
                # Δθ = hrhc.successor_states[:,:,3] - pos.θ # Δθ with opponent at origin
                # pts = [hrhc.car_length hrhc.car_length -hrhc.car_length -hrhc.car_length;
                #     -hrhc.car_width hrhc.car_width hrhc.car_width -hrhc.car_width]/2.0
                # pX = zeros(4,size(hrhc.successor_states,1),size(hrhc.successor_states,2))
                # pY = zeros(size(pX))
                # for i in 1:4
                #     pX[i,:,:] = pts[1,i]*cos(Δθ) - pts[2,i]*sin(Δθ) + ΔX
                #     pY[i,:,:] = pts[1,i]*sin(Δθ) + pts[2,i].*cos(Δθ) + ΔY
                # end

                # collisionFlag = (sum((abs(pX) .< hrhc.car_length/2.0),1)[1,:,:]).*(sum((abs(pY) .< hrhc.car_width/2.0),1)[1,:,:])
                # collisionCost = .001+(collisionFlag .>= 1)./(minimum(abs(pX),1)[1,:,:].*minimum(abs(pY),1)[1,:,:])
            end
        end
    end
end

objective = cost_t + cost_ϕ + cost_tϕ + 1 - dS + collisionCost + Inf * eligibility_mask


In [ ]:
models[2].h

In [ ]:
steps = 1
for _ in 1:steps
    for k_level in 0:maximum([model.k for (id,model) in models])
        for (i, veh) in enumerate(scene)
            model = models[veh.def.id]
            observe!(model, scene, roadway, veh.def.id, tree, obstacleMap, k_level)
            actions[i] = rand(model)
        end
    end
    #     tick!(scene, roadway, actions, models)
    for (veh, action) in zip(scene, actions)
        model = models[veh.def.id]
        context = action_context(model)
        veh.state = propagate(veh, action, context, roadway)
    end
end

k_level = 1
hrhc = models[1]

state = scene.vehicles[hrhc.car_ID].state
hrhc.curve_ind = state.posF.roadind.ind.i

v = state.v # current v
hrhc.v = v

trajectory = zeros(hrhc.h,5)
out_of_bounds = false
objective = zeros(6,17)
eligibility_mask = zeros(6,17)
s = zeros(6,17)
t = zeros(6,17)
ϕ = zeros(6,17)
action_selected = false
cmd = (1,1)
abs_cmd = (1,1)

i = 0
for i in 0:(hrhc.h-1)
    if action_selected
#         @show i-1
        break # out of for loop
    end

    # get legal (reachable from current v, δ) actions
    getLegalMoves!(hrhc, scene, h=hrhc.h-i)

    # calculate successor states
    getSuccessorStates!(hrhc, scene)

    # project successor states onto track
    s,t,ϕ = loopProjectionKD(hrhc, scene, roadway, tree)

    # optimization objective        
    objective = calculateObjective(hrhc, scene, roadway, tree, s, t, ϕ, obstacleMap, k_level, hrhc.h-i)

    while (action_selected==false) && (minimum(objective) != Inf)            
        index = indmin(objective) # find get a better method of optimizing this
        cmd = ind2sub(s, index)
        abs_cmd = (cmd[1]+hrhc.v_idx_low-1, cmd[2]+hrhc.δ_idx_low-1)

        # compute full trajectory up to horizon
        trajectory = computeTrajectory(hrhc, scene, abs_cmd, h=hrhc.h-i)

        # screen trajectory for collisions / validity
        out_of_bounds = screenTrajectory(trajectory, obstacleMap, scene, roadway, hrhc, tree, k_level)

        if out_of_bounds
            objective[index] = Inf
        else
            action_selected=true
            updateObstacleMap!(obstacleMap, k_level, hrhc.car_ID, trajectory)
        end
    end 
end

# hrhc.δ = hrhc.δ_cmds[abs_cmd[1], abs_cmd[2]]
# hrhc.v = hrhc.v_cmds[abs_cmd[1], abs_cmd[2]]

# next_state = VehicleState(VecSE2(trajectory[1,:]),roadway,hrhc.v)   
# hrhc.action = NextState(next_state) # action
plotHRHCInfo(hrhc,models,scene,roadway,s,t,ϕ,objective)

In [ ]:
# plot_stϕ(hrhc,roadway,scene,xP,yP,θP,trajectory,s,t,ϕ,objective)

In [ ]:
f_ϕ=0.0
f_t=.1
f_tϕ=3.0
plotObjectiveHorizon(hrhc,scene,roadway,tree,xP,yP,θP)

In [ ]:
k_level

In [ ]:
vehicleMap = generateObstacleMap(scene, models)
vehicleMap[1][1]

In [ ]:
function plotObjectiveHorizon(hrhc,scene,roadway,tree,xP,yP,θP)
    lo=hrhc.curve_ind
    hi = hrhc.curve_ind + Int(1+2*div(hrhc.V_MAX*hrhc.Δt*hrhc.h,hrhc.Δs))
    
    x = zeros(hrhc.h,size(hrhc.successor_states,1),size(hrhc.successor_states,2))
    y = zeros(size(x))
    z = zeros(size(x))
    s = zeros(size(x))
    t = zeros(size(x))
    ϕ = zeros(size(x))
    objective = zeros(size(x))
    
    for i in 1:hrhc.h
        getLegalMoves!(hrhc, scene, h=i)
        getSuccessorStates!(hrhc, scene)
        x[i,:,:] = copy(hrhc.successor_states[:,:,1])
        y[i,:,:] = copy(hrhc.successor_states[:,:,2])
        z[i,:,:] = copy(hrhc.successor_states[:,:,3])
        s[i,:,:], t[i,:,:], ϕ[i,:,:] = loopProjectionKD(hrhc,scene,roadway,tree)
        objective[i,:,:] = calculateObjective(hrhc,scene, roadway, tree,s[i,:,:],t[i,:,:],ϕ[i,:,:],obstacleMap,hrhc.k,hrhc.h)
    end
    
    PyPlot.figure(figsize=[12,4])

    PyPlot.subplot(141) # ϕ
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    PyPlot.scatter(x,y,c=ϕ,edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("|phi|")
    
    PyPlot.subplot(142) # s
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    PyPlot.scatter(x,y,c=s,edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("s")
    
    PyPlot.subplot(143) # t
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    PyPlot.scatter(x,y,c=t,edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("t")
    
    PyPlot.subplot(144) # objective
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    PyPlot.scatter(x,y,c=log(objective),edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("log objective")
    
end

In [ ]:
function plot_stϕ(hrhc,roadway,scene,xP,yP,θP,trajectory,s,t,ϕ,objective)
    lo=hrhc.curve_ind
    hi=hrhc.curve_ind + Int(1+2*div(hrhc.V_MAX*hrhc.Δt*hrhc.h,hrhc.Δs))
    if hi > length(roadway.segments[1].lanes[1].curve)
        lo = length(roadway.segments[1].lanes[1].curve)
        hi=hrhc.curve_ind + Int(1+2*div(hrhc.V_MAX*hrhc.Δt*hrhc.h,hrhc.Δs))
    end
    
    PyPlot.figure(figsize=[12,4])
    PyPlot.subplot(141)
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    # PyPlot.scatter(Pts[1,:],Pts[2,:],color="red")
    PyPlot.scatter(hrhc.successor_states[:,:,1],hrhc.successor_states[:,:,2],c=abs(ϕ),edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.scatter(scene.vehicles[hrhc.car_ID].state.posG.x, scene.vehicles[hrhc.car_ID].state.posG.y, c="k", edgecolors="none",s=40)
    PyPlot.axis("off")
    PyPlot.title("|phi|")

    PyPlot.subplot(142)
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    # PyPlot.scatter(Pts[1,:],Pts[2,:],color="red")
    PyPlot.scatter(hrhc.successor_states[:,:,1],hrhc.successor_states[:,:,2],c=abs(t),edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.scatter(scene.vehicles[hrhc.car_ID].state.posG.x, scene.vehicles[hrhc.car_ID].state.posG.y, c="k", edgecolors="none",s=40)
    PyPlot.axis("off")
    PyPlot.title("|t|")

    PyPlot.subplot(143)
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    # PyPlot.scatter(Pts[1,:],Pts[2,:],color="red")
    PyPlot.scatter(hrhc.successor_states[:,:,1],hrhc.successor_states[:,:,2],c=s,edgecolor="none")
    PyPlot.scatter(scene.vehicles[hrhc.car_ID].state.posG.x, scene.vehicles[hrhc.car_ID].state.posG.y, c="k", edgecolors="none",s=40)
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("s")


    PyPlot.subplot(144)
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    # PyPlot.scatter(Pts[1,:],Pts[2,:],color="red")
    PyPlot.scatter(hrhc.successor_states[:,:,1],hrhc.successor_states[:,:,2],c=log(objective),edgecolor="none")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")    
    PyPlot.scatter(scene.vehicles[hrhc.car_ID].state.posG.x, scene.vehicles[hrhc.car_ID].state.posG.y, c="k", edgecolors="none",s=40)
    PyPlot.axis("off")
    PyPlot.title("objective")
end

In [ ]:
function plotHRHCInfo(hrhc,models,scene,roadway,s,t,ϕ,objective)
    lo=hrhc.curve_ind
    hi = hrhc.curve_ind + Int(1+2*div(hrhc.V_MAX*hrhc.Δt*hrhc.h,hrhc.Δs))
    lo, hi
    PyPlot.figure(figsize=[12,10])
    # Plot Raceway
    PyPlot.subplot(221)
    # plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    plotSplineRoadway(xP,yP,θP,lane_width)
    PyPlot.scatter(hrhc.successor_states[:,:,1],hrhc.successor_states[:,:,2],color="red")
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.scatter(roadway.segments[1].lanes[1].curve[hrhc.curve_ind].pos.x, roadway.segments[1].lanes[1].curve[hrhc.curve_ind].pos.y, c="white", s=40)
    for (id,car) in models
        if id != hrhc.car_ID
            PyPlot.scatter(scene.vehicles[id].state.posG.x,scene.vehicles[id].state.posG.y,c="blue",s=40)
        end
    end
    PyPlot.axis("off")
    PyPlot.title("Raceway with Motion Primitives")

    PyPlot.subplot(222)
    plotSplineRoadway(xP[lo:hi],yP[lo:hi],θP[lo:hi],lane_width)
    PyPlot.scatter(scene.vehicles[hrhc.car_ID].state.posG.x, scene.vehicles[hrhc.car_ID].state.posG.y, c="red", edgecolors="none",s=40)
    PyPlot.scatter(hrhc.successor_states[:,:,1], hrhc.successor_states[:,:,2],c=log(objective),edgecolors="none")
    PyPlot.scatter(hrhc.successor_states[cmd[1],cmd[2],1], hrhc.successor_states[cmd[1],cmd[2],2],c="white",s=40)
    PyPlot.plot(trajectory[:,1],trajectory[:,2],color="red")
    PyPlot.axis("off")
    PyPlot.title("Log Objective Function")
end

## Visualization of Objective function

In [ ]:
# dS = s_grid - state.s # find increase in arc length
# dS = dS/(maximum(dS))
# s_objective = 1 - dS

h = 10

ϕMAX = 1 # Float64(π)/2
tMAX = 1.0 # roadway.segments[1].lanes[1].width
ϕ_steps = 100
t_steps = 100

ϕgrid = linspace(-ϕMAX,ϕMAX,ϕ_steps)*ones(1,t_steps)
tgrid = (linspace(-tMAX,tMAX,t_steps)*ones(1,ϕ_steps))'
# cost_function1 = (10*(tgrid.^2)/(tMAX.^2) + 2*(ϕgrid.^2)/(ϕMAX.^2))/12
f_ϕ = 0 # 2
f_t = .5 # 8
f_tϕ = 1 # 5

A = [1 1;
     1 1]
cost_function_cross = (ϕgrid*A[1,1] + tgrid*A[2,1]).*(ϕgrid) + (ϕgrid*A[1,2] + tgrid*A[2,2]).*(tgrid)
PyPlot.figure(figsize=[12,7])
for i in 0:9
    h = 10-i
    PyPlot.subplot(2,5,i+1)
    cost_function1 = (exp(((10-h+f_t)*abs(tgrid).^2)) - 1)/exp(f_t)
    cost_function2 = (exp(((f_ϕ)*abs(ϕgrid).^2)) - 1)/exp(f_ϕ)
    cost_function3 = (exp(f_tϕ*cost_function_cross) - 1)/exp(1)
    cost_function = cost_function1+cost_function2+cost_function3
#     PyPlot.plot(cost_function)
    PyPlot.pcolor(cost_function.*(cost_function .< 1.0))
    PyPlot.axis("off")
    PyPlot.title("h = $h")
end